In [55]:
import qiskit as qk
import numpy as np
from copy import deepcopy
from qiskit.circuit.random import random_circuit

In [56]:
np.random.seed(42)

n_qubits = 4
depth = 20
circuit = random_circuit(n_qubits,depth)

In [57]:
print(len(circuit))

50


In [58]:
class ansatz:
    def __init__(self,n_qubits,reps):
        self.n_params = 2*reps*n_qubits
        self.n_qubits = n_qubits
        self.reps = reps
    
    def __call__(self,circuit,theta,inverse=False):
        i = 0
        q_reg = circuit.qregs[0]
        circ = qk.QuantumCircuit(q_reg)
        for rep in range(self.reps):
            for qubit in range(self.n_qubits):
                circ.ry(theta[i],q_reg[qubit])
                i+=1
                circ.rz(theta[i],q_reg[qubit])
                i+= 1
            for qubit in range(self.n_qubits-1):
                circ.cx(q_reg[qubit],q_reg[qubit+1])
                
        if inverse:
            circ = circ.inverse()
        return(circuit+circ)
    
class ansatz2:
    def __init__(self,n_qubits,reps):
        self.n_params = (reps+1)*n_qubits
        self.n_qubits = n_qubits
        self.reps = reps
    
    def __call__(self,circuit,theta,inverse=False):
        i = 0
        q_reg = circuit.qregs[0]
        circ = qk.QuantumCircuit(q_reg)
        circ.ry(theta[i],q_reg[0])
        i+=1
        circ.ry(theta[i],q_reg[1])
        i+=1
        circ.ry(theta[i],q_reg[2])
        i+=1
        circ.ry(theta[i],q_reg[3])
        i+=1
        for rep in range(self.reps):
            for qubit in range(0, self.n_qubits, 2):
                circ.cx(q_reg[qubit], q_reg[qubit+1])
                circ.rz(theta[i],q_reg[qubit+1])
                circ.cx(q_reg[qubit], q_reg[qubit+1])
                i+=1
            for qubit in range(1, self.n_qubits-1, 2):
                circ.cx(q_reg[qubit], q_reg[qubit+1])
                circ.rz(theta[i],q_reg[qubit+1])
                circ.cx(q_reg[qubit], q_reg[qubit+1])
                i+=1
            circ.cx(q_reg[0], q_reg[-1])
            circ.rz(theta[i], q_reg[-1])
            circ.cx(q_reg[0], q_reg[-1])
            i+=1
            
        if inverse:
            circ = circ.inverse()
        return(circuit+circ)
    
class ansatz3:
    def __init__(self,n_qubits,reps):
        self.n_params = 2*reps*n_qubits
        self.n_qubits = n_qubits
        self.reps = reps
    
    def __call__(self,circuit,theta,inverse=False):
        i = 0
        q_reg = circuit.qregs[0]
        circ = qk.QuantumCircuit(q_reg)
        for rep in range(self.reps):
            for qubit in range(self.n_qubits):
                circ.ry(theta[i],q_reg[qubit])
                i+=1
                
            for qubit in range(0, self.n_qubits, 2):
                circ.cx(q_reg[qubit], q_reg[qubit+1])
                circ.rz(theta[i],q_reg[qubit+1])
                circ.cx(q_reg[qubit], q_reg[qubit+1])
                i+=1
                
            for qubit in range(1, self.n_qubits-1, 2):
                circ.cx(q_reg[qubit], q_reg[qubit+1])
                circ.rz(theta[i],q_reg[qubit+1])
                circ.cx(q_reg[qubit], q_reg[qubit+1])
                i+=1
                
            circ.cx(q_reg[0], q_reg[-1])
            circ.rz(theta[i], q_reg[-1])
            circ.cx(q_reg[0], q_reg[-1])
            i+=1
    
        if inverse:
            circ = circ.inverse()
        return(circuit+circ)
    
class Adam():
    def __init__(self, lr=0.01, beta1=0.9, beta2=0.999, eps=1e-8):
        self.lr = lr
        self.beta1 = beta1
        self.beta2 = beta2
        self.eps = eps
        self.m = None
        self.v = None
        self.t = None

    def initialize(self, dims):
        self.m = []
        self.v = []
        self.t = 0

        for dim in dims:
            self.m.append(np.zeros(dim))
            self.v.append(np.zeros(dim))

    def __call__(self, weight_gradient_list):
        self.t += 1
        weight_gradient_modified = []

        for grad, m_, v_ in zip(weight_gradient_list, self.m, self.v):
            m_[:] = self.beta1 * m_ + (1 - self.beta1) * grad
            v_[:] = self.beta2 * v_ + (1 - self.beta2) * grad**2

            m_hat = m_ / (1 - self.beta1**self.t)
            v_hat = v_ / (1 - self.beta2**self.t)
            grad_modified = m_hat / (np.sqrt(v_hat) + self.eps)
            weight_gradient_modified.append(grad_modified)

        return weight_gradient_modified

In [60]:
np.random.seed(42)

class RCO:
    def __init__(self,ansatz,optimizer=None,shots=10000):
        self.n_params = ansatz.n_params
        self.ansatz = ansatz
        self.shots=shots
        self.optimizer=optimizer

    def divide_circuit(self,circuit,divisor):
        circuit_size = len(circuit)
        gates_per_sub_circuit = int(circuit_size/divisor)
        k = 0
        circuit_list = []
        while k < circuit_size:
            circ = deepcopy(circuit)
            for i in range(k):
                circ.data.pop(0)
            for i in range(circuit_size - gates_per_sub_circuit - k):
                circ.data.pop(-1)
            circuit_list.append(deepcopy(circ))
            k+= gates_per_sub_circuit
        return(circuit_list)
    
    def __call__(self,circuit,theta,theta_prev=None):
        
        q_reg = circuit.qregs[0]
        circ = qk.QuantumCircuit(q_reg)
            
            
        circ = self.ansatz(circ,theta)
        circ += circuit.inverse()
        if not theta_prev is None:
            circ = self.ansatz(circ,theta_prev,inverse=True)
        c_reg = qk.ClassicalRegister(q_reg.size)
        circ.add_register(c_reg)
        circ.measure(q_reg,c_reg)
        shots=self.shots
        
        job = qk.execute(circ,
                        backend=qk.Aer.get_backend('qasm_simulator'),
                        shots=shots,
                        )
        results = job.result()
        results = results.get_counts(circ)

        probs = np.zeros(q_reg.size)
        for key,value in results.items():
            key_ = key[::-1]
            for i,bit in enumerate(key_):
                if bit == '1':
                    probs[i] += value
        probs /= shots
        probs = np.mean(probs)
        return(probs)
    
    def get_gradients(self,circuit,theta,theta_prev=None):
        grads = np.zeros(theta.shape)
        for i,param in enumerate(theta):
            theta[i] += np.pi/2
            probs_plus = self(circuit,theta,theta_prev)
            theta[i] -= np.pi
            probs_minus = self(circuit,theta,theta_prev)
            theta[i] += np.pi/2
            grads[i] += (probs_plus - probs_minus)/2
        return(grads)
    
    def gradient_descent(self,circuit,theta,theta_prev=None,lr=0.1,max_iters=1000,tol = 1e-3):
        self.optimizer.initialize([self.ansatz.n_params])
        for epoch in range(max_iters):
            grad = self.get_gradients(circuit,theta,theta_prev)
            grad = self.optimizer([grad])[0]
            theta  = theta - lr*grad
            probs = self(circuit,theta,theta_prev)
            print(epoch,probs)
            if probs <= tol:
                break
        return(theta)
            
    
    def optimize(self,circuit,divisor,lr=0.1,max_iters=1000,tol=1e-3):
        circuit_list = self.divide_circuit(circuit,divisor)
        thetas = np.random.uniform(-np.pi, np.pi, (len(circuit_list), self.n_params))
        for i, circuit in enumerate(circuit_list):
            print(i)
            print(circuit)
            if i != 0:
                thetas[i,:] = np.copy(thetas[i-1,:])
                thetas[i,:] = self.gradient_descent(circuit,thetas[i,:],thetas[i-1,:],lr=lr,max_iters=max_iters,tol=tol)
                
            else:
                thetas[i,:] = self.gradient_descent(circuit,thetas[i,:],lr=lr,max_iters=max_iters,tol=tol)
                
        return(thetas[-1,:])
                   
reps = 4 
circuit_optimizer = RCO(ansatz(n_qubits,reps),optimizer=Adam(), shots=10000)
theta = circuit_optimizer.optimize(circuit,5,tol=1e-3,lr=0.05)

0
     ┌───┐┌───────────────────────────┐┌───┐                        
q_0: ┤ X ├┤ U3(4.2136,0.76276,5.1861) ├┤ H ├────────────────────────
     └─┬─┘└──────────┬─────┬──────────┘├───┤                        
q_1: ──┼─────────────┤ TDG ├───────────┤ X ├──■────■────────────────
       │             └┬───┬┘           └───┘┌─┴─┐  │  ┌────────────┐
q_2: ──■──────────────┤ T ├─────────────────┤ X ├──┼──┤ RX(5.2685) ├
       │             ┌┴───┴┐                └─┬─┘┌─┴─┐└────────────┘
q_3: ──■─────────────┤ SDG ├──────────────────■──┤ Y ├──────────────
                     └─────┘                     └───┘              
0 0.42450000000000004
1 0.37660000000000005
2 0.337075
3 0.3097
4 0.294775
5 0.2818
6 0.2723
7 0.258975
8 0.249275
9 0.24315000000000003
10 0.23225
11 0.21512499999999998
12 0.20392500000000002
13 0.18835
14 0.1713
15 0.157525
16 0.141375
17 0.130625
18 0.12457500000000002
19 0.11134999999999999
20 0.09925
21 0.090075
22 0.08084999999999999
23 0.067825
24 0.05917500000000000

0 0.308275
1 0.264375
2 0.22287500000000002
3 0.203075
4 0.2016
5 0.191625
6 0.186525
7 0.17539999999999997
8 0.16782500000000003
9 0.15745
10 0.153775
11 0.14562499999999998
12 0.13540000000000002
13 0.125825
14 0.11657500000000001
15 0.10995
16 0.10754999999999999
17 0.096825
18 0.09230000000000001
19 0.09285
20 0.087875
21 0.08240000000000001
22 0.08205000000000001
23 0.0793
24 0.07394999999999999
25 0.07505
26 0.067075
27 0.063525
28 0.0644
29 0.06275
30 0.06207499999999999
31 0.060700000000000004
32 0.060025
33 0.05955
34 0.057775
35 0.05605
36 0.05125
37 0.0506
38 0.046675
39 0.04595
40 0.041525
41 0.040299999999999996
42 0.0391
43 0.034175000000000004
44 0.030575
45 0.030225000000000002
46 0.025275
47 0.026475
48 0.024499999999999997
49 0.020675
50 0.022375
51 0.017675000000000003
52 0.017050000000000003
53 0.0129
54 0.012074999999999999
55 0.01395
56 0.011575
57 0.009949999999999999
58 0.01215
59 0.011175000000000001
60 0.009850000000000001
61 0.008425
62 0.009525
63 0.00930000

In [61]:
ans = ansatz(n_qubits,reps)
circ = ans(circuit,theta,inverse=True)

q_reg = circ.qregs[0]
c_reg = qk.ClassicalRegister(q_reg.size)
circ.add_register(c_reg)
circ.measure(q_reg,c_reg)
shots=10000

job = qk.execute(circ,
                backend=qk.Aer.get_backend('qasm_simulator'),
                shots=shots,
                )
results = job.result()
results = results.get_counts(circ)

probs = np.zeros(q_reg.size)
for key,value in results.items():
    print(key,value)
    key_ = key[::-1]
    for i,bit in enumerate(key_):
        if bit == '1':
            probs[i] += value
probs /= shots
print(probs)

0000 9904
0001 5
0010 26
0011 2
0100 3
0101 3
0110 14
0111 5
1001 17
1010 1
1011 2
1100 11
1110 4
1111 3
[0.0037 0.0057 0.0043 0.0038]


In [62]:
np.random.seed(42)

class RCO:
    def __init__(self,ansatz,optimizer=None,shots=10000):
        self.n_params = ansatz.n_params
        self.ansatz = ansatz
        self.shots=shots
        self.optimizer=optimizer

    def divide_circuit(self,circuit,divisor):
        circuit_size = len(circuit)
        gates_per_sub_circuit = int(circuit_size/divisor)
        k = 0
        circuit_list = []
        while k < circuit_size:
            circ = deepcopy(circuit)
            for i in range(k):
                circ.data.pop(0)
            for i in range(circuit_size - gates_per_sub_circuit - k):
                circ.data.pop(-1)
            circuit_list.append(deepcopy(circ))
            k+= gates_per_sub_circuit
        return(circuit_list)
    
    def __call__(self,circuit,theta,theta_prev=None):
        
        q_reg = circuit.qregs[0]
        circ = qk.QuantumCircuit(q_reg)
            
            
        circ = self.ansatz(circ,theta)
        circ += circuit.inverse()
        if not theta_prev is None:
            circ = self.ansatz(circ,theta_prev,inverse=True)
        c_reg = qk.ClassicalRegister(q_reg.size)
        circ.add_register(c_reg)
        circ.measure(q_reg,c_reg)
        shots=self.shots
        
        job = qk.execute(circ,
                        backend=qk.Aer.get_backend('qasm_simulator'),
                        shots=shots,
                        )
        results = job.result()
        results = results.get_counts(circ)

        probs = np.zeros(q_reg.size)
        for key,value in results.items():
            key_ = key[::-1]
            for i,bit in enumerate(key_):
                if bit == '1':
                    probs[i] += value
        probs /= shots
        probs = np.mean(probs)
        return(probs)
    
    def get_gradients(self,circuit,theta,theta_prev=None):
        grads = np.zeros(theta.shape)
        for i,param in enumerate(theta):
            theta[i] += np.pi/2
            probs_plus = self(circuit,theta,theta_prev)
            theta[i] -= np.pi
            probs_minus = self(circuit,theta,theta_prev)
            theta[i] += np.pi/2
            grads[i] += (probs_plus - probs_minus)/2
        return(grads)
    
    def gradient_descent(self,circuit,theta,theta_prev=None,lr=0.1,max_iters=1000,tol = 1e-3):
        self.optimizer.initialize([self.ansatz.n_params])
        for epoch in range(max_iters):
            grad = self.get_gradients(circuit,theta,theta_prev)
            grad = self.optimizer([grad])[0]
            theta  = theta - lr*grad
            probs = self(circuit,theta,theta_prev)
            print(epoch,probs)
            if probs <= tol:
                break
        return(theta)
            
    
    def optimize(self,circuit,divisor,lr=0.1,max_iters=1000,tol=1e-3):
        circuit_list = self.divide_circuit(circuit,divisor)
        thetas = np.random.uniform(-np.pi, np.pi, (len(circuit_list), self.n_params))
        for i, circuit in enumerate(circuit_list):
            print(i)
            print(circuit)
            if i != 0:
                #thetas[i,:] = np.copy(thetas[i-1,:])
                thetas[i,:] = self.gradient_descent(circuit,thetas[i,:],thetas[i-1,:],lr=lr,max_iters=max_iters,tol=tol)
                
            else:
                thetas[i,:] = self.gradient_descent(circuit,thetas[i,:],lr=lr,max_iters=max_iters,tol=tol)
                
        return(thetas[-1,:])
                   
reps = 4 
circuit_optimizer = RCO(ansatz(n_qubits,reps),optimizer=Adam(), shots=10000)
theta = circuit_optimizer.optimize(circuit,5,tol=1e-3,lr=0.05)

0
     ┌───┐┌───────────────────────────┐┌───┐                        
q_0: ┤ X ├┤ U3(4.2136,0.76276,5.1861) ├┤ H ├────────────────────────
     └─┬─┘└──────────┬─────┬──────────┘├───┤                        
q_1: ──┼─────────────┤ TDG ├───────────┤ X ├──■────■────────────────
       │             └┬───┬┘           └───┘┌─┴─┐  │  ┌────────────┐
q_2: ──■──────────────┤ T ├─────────────────┤ X ├──┼──┤ RX(5.2685) ├
       │             ┌┴───┴┐                └─┬─┘┌─┴─┐└────────────┘
q_3: ──■─────────────┤ SDG ├──────────────────■──┤ Y ├──────────────
                     └─────┘                     └───┘              
0 0.42787499999999995
1 0.37422500000000003
2 0.33925
3 0.31605
4 0.29725
5 0.28042500000000004
6 0.27197499999999997
7 0.256975
8 0.25
9 0.24215
10 0.228225
11 0.218275
12 0.2048
13 0.190725
14 0.17679999999999998
15 0.1589
16 0.14750000000000002
17 0.13190000000000002
18 0.12212499999999998
19 0.112725
20 0.10105
21 0.09045
22 0.07752500000000001
23 0.0665
24 0.05985
25 0.

40 0.141625
41 0.132125
42 0.12955
43 0.11685000000000001
44 0.10925
45 0.100575
46 0.09542500000000001
47 0.08715
48 0.07665
49 0.07155
50 0.06697499999999999
51 0.063675
52 0.059975
53 0.055625
54 0.050074999999999995
55 0.04717500000000001
56 0.041125
57 0.038224999999999995
58 0.03635
59 0.032174999999999995
60 0.03275
61 0.03025
62 0.029375
63 0.0288
64 0.024975000000000004
65 0.025975
66 0.026975
67 0.02595
68 0.0257
69 0.02645
70 0.0268
71 0.02195
72 0.021724999999999998
73 0.02085
74 0.019700000000000002
75 0.018475
76 0.017125
77 0.0178
78 0.016025
79 0.015625
80 0.0167
81 0.015675
82 0.014849999999999999
83 0.015349999999999999
84 0.012725
85 0.0123
86 0.013575
87 0.013275000000000002
88 0.012924999999999999
89 0.013474999999999999
90 0.0115
91 0.013
92 0.011574999999999998
93 0.012074999999999999
94 0.0121
95 0.01185
96 0.011425000000000001
97 0.011349999999999999
98 0.011825
99 0.01025
100 0.010074999999999999
101 0.01085
102 0.009999999999999998
103 0.0093
104 0.0091
105 0

90 0.005125
91 0.005125
92 0.00605
93 0.0042250000000000005
94 0.0043
95 0.004
96 0.004525
97 0.003875
98 0.00425
99 0.003825
100 0.0031750000000000003
101 0.003075
102 0.0027
103 0.0043
104 0.002475
105 0.003275
106 0.0032
107 0.002625
108 0.0034500000000000004
109 0.0027750000000000006
110 0.0033
111 0.002175
112 0.002525
113 0.00295
114 0.002375
115 0.0023250000000000002
116 0.00185
117 0.0021
118 0.001975
119 0.0016
120 0.0015249999999999999
121 0.00195
122 0.001425
123 0.0015
124 0.001275
125 0.0016500000000000002
126 0.0019249999999999998
127 0.001425
128 0.0018999999999999998
129 0.0017000000000000001
130 0.0014
131 0.000675
4
     ┌───┐    ┌───┐                             ┌────────────┐               
q_0: ┤ X ├────┤ H ├─────────────────────■───────┤ RY(4.8423) ├───────────────
     └─┬─┘┌───┴───┴────┐                │       └────────────┘    ┌─────┐    
q_1: ──┼──┤ RY(1.5961) ├─■──────────────┼────────■────────────────┤ TDG ├────
       │  └────────────┘ │3.5005 ┌──────┴─────

In [63]:
ans = ansatz(n_qubits,reps)
circ = ans(circuit,theta,inverse=True)

q_reg = circ.qregs[0]
c_reg = qk.ClassicalRegister(q_reg.size)
circ.add_register(c_reg)
circ.measure(q_reg,c_reg)
shots=10000

job = qk.execute(circ,
                backend=qk.Aer.get_backend('qasm_simulator'),
                shots=shots,
                )
results = job.result()
results = results.get_counts(circ)

probs = np.zeros(q_reg.size)
for key,value in results.items():
    print(key,value)
    key_ = key[::-1]
    for i,bit in enumerate(key_):
        if bit == '1':
            probs[i] += value
probs /= shots
print(probs)

0000 9888
0001 16
0011 4
0100 5
0101 3
0110 3
0111 13
1000 6
1001 4
1010 4
1011 16
1100 16
1101 7
1110 15
[0.0063 0.0055 0.0062 0.0068]
